# Visualizing

We often want to visualize one or more exoplanet populations in fairly standard ways. Here we summarize some predefined visualizations for populations and explain how you can create your own multi-panel, multi-population visualizations with `exoatlas`.

In [ ]:
from exoatlas import * 
version() 

We'll be making lots of plots here, so let's give them a wide aspect ratio, so they don't take up too much space.

In [ ]:
plt.rcParams['figure.figsize'] = (8,3)

Let's generate some populations to visualize.

In [ ]:
exoplanets = TransitingExoplanets()
solar = SolarSystem()

## Make Your Own Plots with `exoatlas` Data

It is, of course, possible to make your own plots using data from `exoatlas` populations. You probably have some brilliant idea, and just working with the raw quantities might be where you want to start. Here's a basic example.

In [ ]:
# plot the exoplanets 
x = exoplanets.relative_insolation()
y = exoplanets.radius()
plt.scatter(x, y, marker='.', s=5, alpha=0.5)

# plot the Solar System planets 
x = solar.relative_insolation()
y = solar.radius()
plt.scatter(x, y, marker='s', s=30, color='black')

# adjust the plotting details
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Bolometric Flux (relative to Earth)')
plt.ylabel('Planet Radius (Earth radii)')
plt.xlim(1e5, 1e-5);

In [ ]:
# plot the exoplanets with uncertainties
x = exoplanets.relative_insolation()
y = exoplanets.radius()
x_error = exoplanets.relative_insolation_uncertainty_lowerupper()
y_error = exoplanets.radius_uncertainty_lowerupper()
plt.errorbar(x, y, xerr=x_error, yerr=y_error, linewidth=0, elinewidth=1, alpha=0.5)

# plot the Solar System planets 
x = solar.relative_insolation()
y = solar.radius()
plt.scatter(x, y, marker='s', s=30, color='black')

# adjust the plotting details
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Bolometric Flux (relative to Earth)')
plt.ylabel('Planet Radius (Earth radii)')
plt.xlim(1e5, 1e-5);

You can build up whatever beautiful, transparent, creative, and useful visualizations you want on your own. 

However, often we may want to fill a panel with multiple different planet populations, and maybe even across multiple linked plots. That can be a little annoying to keep track of, so we tried to add a few shortcuts to make it easier to sets of quantities for groups of populations. 

## Make Visualizations using `Plottable` and `Panel`

We provide two core types of objects to help with visualizing: 
- `Plottable` define basics of how to present data for one quantity 
- `Panel` combines multiple `Plottable` objects together to make a plot

The main advantages of working with these `Plottable` + `Panel` tools might be that:
- they set reasonably useful defaults for many common quantities 
- they can be used to loop over multiple populations, each with its own style 
- they make up some common built-in visualizations that you might want to use or modify

In [ ]:
from exoatlas.visualizations import * 

### 📏 `Plottable` objects prepare data for visualization 📏

To get data ready for visualizing, a `Plottable` will generally define some of the following:
- `source` = where does the quantity come from? This should be the name of a method that's available for all the populations you might want to use. 
- `label` = a human-friendly label. This may appear as axis labels or in figure legends. 
- `scale` = are the data better displayed linearly or logarithmically? This will set the scale for x or y axes, or how colors or sizes are normalized. 
- `lim` = what are reasonable limits? This would set default axis limits, or how colors and sizes are define their minimum and maximum values.
- `**kw` = if `source` is a method that takes keyword arguments (such as `teq(albedo=..., f=...)`), any additional keywords you provide when creating a `Plottable` will get passed along to the method

Let's create a few:

In [ ]:
# specify everything 
radius = Plottable(source='radius', 
                   label='Planet Radius (Earth radii)', 
                   scale='log', 
                   lim=[0.3, 30], 
                   unit=u.Rearth)

# leave scale and limits as None, for auto-scaling
distance = Plottable(source='distance', 
                     label='Distance (pc)')

# define scale and limits, but don't worry about a fancy label
teff = Plottable(source='stellar_teff', scale='log', lim=[2500, 7500])

# pass keyword "wavelength" to quantity method 
brightness = Plottable(source='stellar_brightness',
                       scale='linear', lim=[1e3, 1e9],
                       wavelength=1*u.micron)


We indicate these variable as `Plottable` by a little ruler 📏, indicating each is ready to draw some data at the right locations on a plot. 

In [ ]:
radius, distance, brightness, teff

For a given population, it can retrieve values and symmetric or asymmetric uncertainties.

In [ ]:
radius.value(exoplanets)

In [ ]:
radius.uncertainty(exoplanets)

In [ ]:
radius.uncertainty_lowerupper(exoplanets)

We can also calculate normalized values, which may be useful for representing sizes or colors. The normalization will pay attention to the `scale` and `lim` keywords. 

In [ ]:
brightness.value(exoplanets)

In [ ]:
brightness.normalized_value(exoplanets)

If we wanted to stop here, we could use these three `Plottable` objects to help create a plot. Practically, this isn't much different from just making the plot ourselves from the raw data; basically it's just the size normalization that's helping. 

In [ ]:
mass_limit =  0.5*u.Msun
is_lowmass = exoplanets.stellar_mass() < mass_limit
highmass = exoplanets[is_lowmass == False]
highmass.label = f'> {mass_limit}'
lowmass = exoplanets[is_lowmass]
lowmass.label = f'< {mass_limit}'

plt.figure()
for pop in [highmass, lowmass]:
    x = distance.value(pop)
    y = radius.value(pop)
    s = brightness.normalized_value(pop)*1000
    plt.scatter(x, y, s=s, label=pop.label)
plt.xscale('log')
plt.yscale('log')
plt.xlabel(distance.label)
plt.ylabel(radius.label)
plt.legend(frameon=False, loc='upper left', bbox_to_anchor=(1,1));


### 🖼️ `Panel` draws plots with `Plottable` objects 🖼️

With a `Panel`, we can combine a few `Plottable` objects together to build up a plot. The `Panel` is responsible for:
- managing the figure and axes where data will be drawn
- looping over populations and representing them 
- serving as a building block for multi-panel linked visualizations

#### `BubblePanel` for x, y, size, color

For basic scatter plots, we might try `BubblePanel`, where the four ways we might represent data are: 
- `xaxis` = bubble position along the x-axis 
- `yaxis` = bubble position along the y-axis 
- `size` = bubble area, based on `normalized_value`
- `color` = bubble color, based on `normalized_value`, according to a colormap


Let's try this with a basic example. We create a `Panel` (🖼️) from two `Plottable` (📏) objects. We can use this `Panel` to plot individual populations one-by-one with `plot()`...

In [ ]:
panel = BubblePanel(xaxis=distance, yaxis=radius)
panel.plot(highmass)
panel.plot(lowmass)
plt.legend();

 ...or use `build()` to build up the plot by looping over populations.

In [ ]:
panel = BubblePanel(xaxis=distance, yaxis=radius)
panel.build([highmass, lowmass])
plt.legend();

Let's use one more data dimension by having the size represent the brightness of the star as seen from Earth, using color simply to represent the two different populations.

In [ ]:
bubble = BubblePanel(xaxis=distance, yaxis=radius, size=brightness, color=None)
bubble.build([highmass, lowmass])
plt.legend();

Or, if we're focusing primarily on one `Population`, we might use color to represent another quantity. In the plot below, we can see that while stellar brightness at Earth (size) generally increases toward closer distances, stars with cooler stellar effective temperatures (color) have lower intrinsic luminosities and therefore appear less bright, even at nearby distances. 

In [ ]:
bubble = BubblePanel(xaxis=distance, yaxis=radius, size=brightness, color=teff)
bubble.plot(exoplanets)


#### `ErrorPanel` for x, y with uncertainties

Including errorbars on exoplanet population data can get tricky because planets can have wildly heteroscedastic uncertainties. If we just plot errorbars for all data points equally, our eyes are visually drawn to the largest uncertainties, while we'd like them to do the opposite: focus in on the best data! As such, in the `ErrorPanel` we by default scale the intensity of errorbars to visually emphasize the points with the smallest uncertainties. 

In [ ]:
error = ErrorPanel(xaxis=distance, yaxis=radius)
error.plot(exoplanets)

### 🖼️ `Gallery` collects `Panel` together 🖼️

Often, we may want to look at multple plots side-by-side, to see how trends in one view might relate to other properties. A `Gallery` can be built up from a collection of `Panel` objects, like this.

In [ ]:
neat_planet = exoplanets['HD209458b']
neat_planet.color='magenta'
neat_planet.s=400
neat_planet.zorder=1e20
neat_planet.alpha=1
neat_planet.bubble_anyway=True
neat_planet.outlined=True 
neat_planet.filled=False


In [ ]:
row = Gallery() 
row.build_panels([exoplanets, solar, neat_planet])

In [ ]:
physical = physical_summary() 
physical.build_panels([exoplanets, solar, neat_planet])

## Built-in Multipanel Visualizations 
`exoatlas` has a few pre-defined visualizations. The quickest way to dive into visualizing your exoplanet population might be to start with one of these. Below, we show you how to build up your own visualizations from scratch.

In [ ]:
exoplanets.color

In [ ]:
#observable_summary([exoplanets, solar]);

In [ ]:
from exoatlas import * 
from exoatlas.visualizations import * 

class StellarBrightness(Plottable):
    source='stellar_brightness'
    scale = "log"
    lim = [None, None]  # [1e2, 1e8]
    unit = u.Unit(("ph s^-1 m^-2 micron^-1"))

    def __init__(self, **kw):
        """
        Initialize for a particular wavelength, because the
        eclipse depth will depend on the thermal emission
        spectrum of the planet and star.
        """
        Plottable.__init__(self, **kw)

        # set up the units
        self.unit = u.Unit(("ph s^-1 m^-2 micron^-1"))
        self.unit_string = "photons/s/m$^2$/$\mu$m"

        # set the label
        self.label = f'Stellar Brightness at Earth at $\lambda={self.kw['wavelength'].to("micron").value:.1f}\mu$m\n({self.unit_string})'


In [ ]:
e = TransitingExoplanets() 
b = StellarBrightness(wavelength=5*u.micron)
b.value(e), b.label

In [ ]:
b = StellarBrightness(wavelength=1*u.micron)
b.value(e), b.label

In [ ]:
cmap = plt.matplotlib.colormaps['gray']


In [ ]:
cmap([2, 5, 10])

In [ ]:

a = vi.BubblePanel(xaxis=vi.StellarRadius, 
                   yaxis=vi.Radius, 
                   color=vi.Teq,
                   size=vi.StellarBrightness(wavelength=1*u.micron))

b = vi.BubblePanel(xaxis=vi.StellarRadius, 
                   yaxis=vi.Mass, 
                   color=vi.Teq,
                   size=vi.StellarBrightness(wavelength=1*u.micron))

g = vi.Gallery(panels=[a,b], horizontal=False)
g.build_panels([exoplanets])
